<a href="https://colab.research.google.com/github/vivshar/colab/blob/master/time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import keras
import numpy as np

In [0]:
input_file = '/content/jena_climate/jena_climate_2009_2016.csv'
f = open(input_file)
data = f.read()
f.close()

In [0]:
data = data.split('\n')
header = data[0]
header = header.split(',')
data = data[1:]

In [0]:
float_data = np.zeros((len(data),len(header)-1))
for i,line in enumerate(data):
  values = [float(x) for x in line.split(',')[1:]]
  float_data[i] = values

In [0]:
#normalizing
mean = float_data[:20000].mean(axis = 0)
float_data -= mean
std = float_data[:20000].std(axis = 0)
float_data /=std

In [0]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=6):
  i = min_index + lookback
  if shuffle == True:
    rows = np.random.randint(min_index + lookback,max_index,size = batch_size)
  else:
    if i+batch_size >= max_index :
      i = min_index+lookback
    i+=len(rows)
    rows = range(i,min(max_index,i+batch_size))
  sample = np.zeros((batch_size,lookback//step,data.shape[-1]))
  targets = np.zeros((batch_size))
  for j,row in enumerate(rows):
    indices = range(rows[j]-lookback,rows[j],step)
    sample[j] = data[indices]
    targets[j] = data[rows[j]+delay][1]
  yield sample,targets

In [0]:
lookback = 1440
step = 6
delay = 144
batch_size = 128

In [0]:
train_gen = generator(float_data,lookback=lookback,delay=delay,min_index=0,max_index=200000,shuffle=True,step=step,batch_size=batch_size)
val_gen = generator(float_data,lookback=lookback,delay=delay,min_index=200001,max_index=300000,step=step,batch_size=batch_size)
test_gen = generator(float_data,lookback=lookback,delay=delay,min_index=300001,max_index=None,step=step,batch_size=batch_size)

In [0]:

val_steps = (300000 - 200001 - lookback)
test_steps = (len(float_data) - 300001 - lookback)